In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pytorch-msssim

In [3]:
import glob
import random
import os
import numpy as np
import argparse
import math
import itertools
import sys

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

from torch.utils.data import DataLoader
from torch.autograd import Variable

from matplotlib import pyplot as ply

from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM

# Normalization parameters for pre-trained PyTorch models
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

In [4]:
from drive.MyDrive.ImageSuperResolution import SRGAN

In [5]:
class ImageDataset(Dataset):
    def __init__(self, root, shape, transform):
        height, width = shape
        # Transforms for low resolution images and high resolution images
        self.transform = transform

        self.files = sorted(glob.glob(root + "/*.*"))

    def __getitem__(self, index):
        low_path = self.files[index % len(self.files)]
        #high_path = low_path.replace('low','high')
        
        img_low = Image.open(low_path)
        #img_high = Image.open(high_path)
        
        img_low = self.transform(img_low)
        #img_high = self.transform(img_high)
         
        name = low_path[80:]

        return {"low": img_low, "name": name}


    def __len__(self):
        return len(self.files)

In [6]:
root = '/content/drive/MyDrive/ImageSuperResolution/DatasetBatches/test/LLCNN_generated/'
shape = (400,400)

transform = transforms.Compose(
            [
                #transforms.Resize((shape[0], shape[1]), Image.BICUBIC),
                transforms.ToTensor(),
                #transforms.Normalize(mean, std),
            ]
        )

In [7]:
data = ImageDataset(root, shape, transform)

In [8]:
data[0]['low'].shape

torch.Size([3, 100, 100])

In [9]:
data[0]['name']

'image5000.jpg'

In [10]:
n = 200
batch_size = 2
channels = 3

In [11]:
cuda = torch.cuda.is_available()

In [12]:
generator = SRGAN.GeneratorResNet()

In [13]:
if cuda:
    generator = generator.cuda()

In [14]:
generator.load_state_dict(torch.load("/content/drive/MyDrive/ImageSuperResolution/SRGAN/saved_models/generator_"+str(n-1)+".pth"))

<All keys matched successfully>

In [15]:
dataloader = DataLoader(
    data,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
)

In [16]:
Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor

In [18]:
os.makedirs("/content/drive/MyDrive/ImageSuperResolution/DatasetBatches/test/SRGAN_generated", exist_ok=True)

In [19]:
data = iter(dataloader)

In [20]:
next(data)['name']

['image5692.jpg', 'image5310.jpg']

In [ ]:
generated_image_root = 'DatasetBatches/test/SRGAN_generated/'
for i, imgs in enumerate(dataloader):
        img_low = Variable(imgs["low"].type(Tensor))
        outputs = generator(img_low)
        outputs.detach_
        outputs = outputs.to('cpu')
        for i, img in enumerate(outputs):
            save_image(img, generated_image_root + imgs['name'][i])